In [1]:
import numpy as np
from PIL import Image, ImageOps
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
from keras.datasets import mnist

ROWS = COLS = 28

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], ROWS, COLS, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], ROWS, COLS, 1).astype('float32')

Using TensorFlow backend.


In [ ]:
X_train = np.append(X_train, X_test, axis=0)
y_train = np.append(y_train, y_test, axis=0)

X_train = X_train[np.where(y_train!=9)]
y_train = y_train[np.where(y_train!=9)]

In [ ]:
X_train.shape

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
gen = ImageDataGenerator(rotation_range=360, width_shift_range=0.1, height_shift_range=0.1, zoom_range=-0.3)

In [ ]:
new_x_train = []
new_y_train = []
m = len(y_train)

gen = ImageDataGenerator(rotation_range=360, width_shift_range=0.1, height_shift_range=0.1, zoom_range=-0.3)

for i, (y, im) in enumerate(zip(y_train, X_train)):
    im[im<=30] = 0
    im[im>30] = 1
    im = im.reshape(1,28,28,1)
    it = gen.flow(im, batch_size=1)
    n = 10
    new_x_train.append(im)
    new_y_train += [y]*11
    for _ in range(10):
        im = it.next().astype('uint8')
        new_x_train.append(im)
    if (i+1)%100==0: print(f'Handled {i+1}/{m} rows. Generated {(i+1) * 11} images.', end='\r')

In [ ]:
X_train = np.array(new_x_train).reshape(len(new_x_train),28,28,1)

In [ ]:
y_train = np.array(new_y_train)

In [ ]:
np.save('big_X_train', X_train)
np.save('big_y_train', y_train)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train = np.load('big_X_train.npy')
y_train = np.load('big_y_train.npy')

In [ ]:
X_train.shape, y_train.shape

In [ ]:
len(X_train)//len(np.unique(y_train))

In [ ]:
# add = 9
# divide = 10
# subtract = 11
# multiply = 12
# equals = 13

In [ ]:
samples = ['pad_0.png', 'pad_1.png', 'pad_2.png', 'pad_3.png'] 
symbol, alias = ('equals', '13')
n = 77000//len(samples)

for s in samples:
    base_im = Image.open(f'data_gen/{symbol}/{s}')
    base_im = np.array(ImageOps.invert(base_im))
    base_im = base_im.reshape(1, ROWS, COLS, 1).astype('uint8')

    it = gen.flow(base_im, batch_size=1)
    arr = []
    for i in range(n):
        im = it.next().astype('uint8')
        im[im<=30] = 0
        im[im>30] = 1
        arr.append(im)
        if (i+1)%100==0: print(f'Generated {i+1}/{n} train images', end='\r')
    X_train = np.append(X_train, np.array(arr)[:,0,:], axis=0)
    y_train = np.append(y_train, [alias]*n, axis=0)
    print(f'Generated all train images for {s}!         ')

In [ ]:
np.save('super_big_X_train', X_train)
np.save('super_big_y_train', y_train)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
len(X_train) - len(y_train)

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [6]:
batch_size = 128
num_classes = 14
epochs = 1

input_shape = (ROWS, COLS, 1)

In [7]:
X_train = np.load('super_big_X_train.npy')
y_train = np.load('super_big_y_train.npy')

In [8]:
len(y_train) - len(X_train)

0

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [10]:
import keras

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:
model.save('mod_pad_1_epoch')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )

In [ ]:
model.save('mod_pad_2_epochs')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_3_epochs')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_4_epochs')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_5_epochs')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_6_epochs')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_7_epochs')

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_8_epochs')

In [13]:
model = keras.models.load_model('mod_pad_8_epochs')

In [14]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_9_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 2728s 3ms/step - loss: 0.3158 - acc: 0.9069 - val_loss: 0.1645 - val_acc: 0.9507


In [15]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_10_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 2079s 2ms/step - loss: 0.3130 - acc: 0.9079 - val_loss: 0.1701 - val_acc: 0.9502


In [16]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_11_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1845s 2ms/step - loss: 0.3099 - acc: 0.9089 - val_loss: 0.1908 - val_acc: 0.9489


In [17]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_12_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1705s 2ms/step - loss: 0.3089 - acc: 0.9097 - val_loss: 0.1644 - val_acc: 0.9513


In [18]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_13_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1659s 2ms/step - loss: 0.3058 - acc: 0.9103 - val_loss: 0.1761 - val_acc: 0.9494


In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_14_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1288s 1ms/step - loss: 0.3041 - acc: 0.9110 - val_loss: 0.2782 - val_acc: 0.9388


In [20]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          verbose=1,
          validation_data=(X_test, y_test)
         )
model.save('mod_pad_15_epochs')

Train on 862769 samples, validate on 215693 samples
Epoch 1/1
862769/862769 [==============================] - 1227s 1ms/step - loss: 0.3016 - acc: 0.9120 - val_loss: 0.1645 - val_acc: 0.9528


In [ ]:
X_train[0].shape